In [ ]:
import jax
jax.devices()

In [ ]:
from scripts.nj.neurosci import *
import scripts.nj.graph_to_arrays as ga
import scripts.data_preparation as dp
import networkx as nx

## Получаем данные

In [ ]:
neurons_ids = [
    "7055857",
]

DIR = "Ilya/trash/del_this"
neurons_ids = [int(i) for i in neurons_ids]
sc = dp.simulation_context(DIR, neurons_ids)
sc.build_full_graph()
full_g = nx.read_gml(sc.path_to_full_graph)

In [ ]:
from Ilya.trash.graph_to_arrays import DirectednessMap
from scripts.data_preparation import Pexist

In [ ]:
jc = ga.SimulationContextJax(full_g, node_type_groups = {
    'cable':['branch', 'root', 'slab', 'end', 'connector'],
}, edge_directedness={'cable': {'cable': False},}, initial_node_values={
    'cable':1.0
}, cache_dir=DIR + '/jax')

In [ ]:
res = jc.get_context()
res

In [ ]:
local_indeces_mapping = jc.get_node_id_mapping()
local_indeces_mapping

In [ ]:
metadata = sc.node_metadata
metadata

In [ ]:
metadata = metadata.fillna(10.0) # 10.0 as basic radius
metadata['new_index'] = metadata.apply(lambda row:local_indeces_mapping['cable'].get(str(row['node_id'])), axis = 1)
metadata = metadata.dropna(subset=['new_index'])
metadata = metadata.set_index('new_index').sort_index()
metadata

In [ ]:
res['edges_cable_to_cable'].T

## Определяем структуру симуляции

In [ ]:
def get_my_pipeline(constants, dt = 0.1):
    r = jnp.array(metadata['radius'].to_numpy())
    x = jnp.array(metadata['x'].to_numpy())
    y = jnp.array(metadata['y'].to_numpy())
    z = jnp.array(metadata['z'].to_numpy())
    S = np.pi * r**2
    cable_m = res['edges_cable_to_cable'].T

    dx = x.at[cable_m[:, 1]].get() - x.at[cable_m[:, 0]].get()
    dy = y.at[cable_m[:, 1]].get() - y.at[cable_m[:, 0]].get()
    dz = z.at[cable_m[:, 1]].get() - z.at[cable_m[:, 0]].get()

    L = (dx**2 + dy**2 + dz**2)**0.5
    ro = 1.0 # по идеи должно быть 100, но потом разберемся
    R = (ro*L/S.at[cable_m[:, 1]].get())

    integrate = get_euler_step(dt) # получаем функцию для интегрирования
    HH = get_HH_pipeline(**constants) # получаем функцию для HH
    cable = laplace_at_graph_symetric(cable_m, 'V', scaling = R) # получаем функцию для динамики кабелей
    @jax.jit
    def my_pipeline(state):
        s, ds = to_diff(state) # создает ds той же формы что и state, но заполненный нулями
        s, ds = HH(s, ds) # вставляет HH каналы
        s, ds = cable(s, ds) # соединяет сегменты
        #ds['V'] += ds['V'].at[0].add((s['time'] > 20.0) * 0.01*(jnp.sin(s['time']/20.0) + 1.0)/2.0) # внешние стимулы, в данном случае синусоидольный ток с 20 мс
        ds['V'] += ds['V'].at[0].add(2.0*(s['time'] > 20.0))
        s = integrate(s, ds) # интегрируем
        return s
    return my_pipeline


In [ ]:
jnp.array(metadata['radius'].to_numpy())

### Начальные значения и константы


In [ ]:
num_nodes = res['num_nodes']['cable']

initials = {
    "V":jnp.ones((num_nodes, ), jnp.float32)*-65.0,
    "m":jnp.ones((num_nodes, ), jnp.float32)*0.0220,
    'n':jnp.ones((num_nodes, ), jnp.float32)*0.0773,
    'h':jnp.ones((num_nodes, ), jnp.float32)*0.9840,
    "time":0.0
}

consts = {
    "C": 1.0,      # Емкость мембраны (мкФ/см^2)
    "ENa": 50.0,   # Равновесный потенциал Na+ (мВ)
    "EK": -77.0,   # Равновесный потенциал K+ (мВ)
    "EL": -54.4,   # Равновесный потенциал утечки (мВ)
    "gNa": 120.0,  # Максимальная проводимость Na+ (мСм/см^2)
    "gK": 36.0,    # Максимальная проводимость K+ (мСм/см^2)
    "gL": 0.3,     # Проводимость утечки (мСм/см^2),
}


In [ ]:
my_pipeline = get_my_pipeline(consts, 0.0001)

## Запуск симуляции

In [ ]:
jsim = simulation(initials, my_pipeline, 2000)
H = jsim.run(100)

## Построение графиков

In [ ]:
import matplotlib.pyplot as plt
t, v = H['time'], H['V']
t, v = np.array(t), np.array(v)
plt.plot(v)
plt.show()

In [ ]:
v